In [ ]:
# !pip install recommenders
# !pip install tf_slim

In [ ]:
import sys
import os
import shutil
# import papermill as pm
# import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Pandas version: 1.3.5
Tensorflow version: 2.8.2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
from sklearn.metrics.pairwise import cosine_similarity
import ast
import os
from sklearn.preprocessing import MinMaxScaler
import pickle
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = '/content/drive/MyDrive/쿠아이/쿠아이 컨퍼런스/2022 하계/'

In [ ]:
interaction = pd.read_csv(path + 'Grocery_and_Gourmet_Food_over_30_5_interaction.csv')
meta = pd.read_csv(path + 'FINAL_META_DATA.csv')
final_df = pd.read_csv(path + 'grocery_final.csv')

In [ ]:
category_sim = pd.read_csv(path + 'category_cos_sim_by_asin.csv')
category_sim.set_index('Unnamed: 0', inplace=True)

In [ ]:
description_sim = pd.read_csv(path + 'description_cos_sim_by_asin.csv')
description_sim.set_index('Unnamed: 0', inplace=True)

# preprocessing

In [ ]:
data = final_df[['reviewerID', 'asin', 'overall', 'unixReviewTime']]

In [ ]:
data.rename({'reviewerID':'userID', 'asin':'itemID', 'overall':'rating', 'unixReviewTime':'timestamp'}, axis=1, inplace=True)

In [ ]:
train, test = python_chrono_split(data, 0.8)

In [ ]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [ ]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [ ]:
data_pivot = data.pivot_table('rating', index = 'userID', columns='itemID')

In [ ]:
data_pivot.fillna(0, inplace=True)
data_pivot.head()

itemID,4639725043,9742356831,B00005BPQ9,B00006BN4U,B00006FMLY,B00006IDK9,B00008RCN8,B0000CEO8F,B0000CFPI2,B0000CH4FT,...,B01GSW9U5Y,B01GSWBVNS,B01GT3KT52,B01H0AI5JK,B01H3VFR6U,B01H3VFVT8,B01H4G38JM,B01H6IQING,B01H6JO1EI,B01HCV88MQ
userID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100UD67AHFODS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100ZQDV7L8PVV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
A1047EDJ84IMAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_sim = cosine_similarity(data_pivot.T)

In [ ]:
item_sim_df = pd.DataFrame(data=item_sim, index=data_pivot.columns,
                          columns=data_pivot.columns)
print(item_sim_df.shape)

(6186, 6186)


In [ ]:
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [ ]:
ratings_pred = predict_rating(data_pivot.values , item_sim_df.values)
ratings_pred_matrix_i = pd.DataFrame(data=ratings_pred, index= data_pivot.index,
                                   columns = data_pivot.columns)

In [ ]:
ratings_pred_matrix_i.head(3)

itemID,4639725043,9742356831,B00005BPQ9,B00006BN4U,B00006FMLY,B00006IDK9,B00008RCN8,B0000CEO8F,B0000CFPI2,B0000CH4FT,...,B01GSW9U5Y,B01GSWBVNS,B01GT3KT52,B01H0AI5JK,B01H3VFR6U,B01H3VFVT8,B01H4G38JM,B01H6IQING,B01H6JO1EI,B01HCV88MQ
userID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,0.000000,0.027247,0.018410,0.089756,0.045278,0.006124,0.035796,0.020929,0.104250,0.000000,...,0.015804,0.037217,0.000000,0.000000,0.012807,0.008420,0.102057,0.029454,0.000000,0.000000
A100UD67AHFODS,0.030989,0.027753,0.023077,0.060320,0.028144,0.000000,0.029956,0.018504,0.024202,0.020953,...,0.017368,0.013271,0.003455,0.001991,0.017977,0.022433,0.000000,0.018922,0.017327,0.052678
A100WO06OQR8BQ,0.039995,0.033580,0.057587,0.059609,0.053330,0.089757,0.052074,0.048201,0.027471,0.081454,...,0.025653,0.074212,0.147738,0.031869,0.091485,0.060576,0.076471,0.047067,0.107133,0.027405


# mf rating

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [ ]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [ ]:
P, Q = matrix_factorization(data_pivot.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  4.542202956601146
### iteration step :  10  rmse :  0.7992062032000317
### iteration step :  20  rmse :  0.6482833176392826
### iteration step :  30  rmse :  0.4848404784102301
### iteration step :  40  rmse :  0.34099153872031535
### iteration step :  50  rmse :  0.24143027331274206
### iteration step :  60  rmse :  0.17741357403394242
### iteration step :  70  rmse :  0.13605143565861255
### iteration step :  80  rmse :  0.10851580674970146
### iteration step :  90  rmse :  0.08961522430683566
### iteration step :  100  rmse :  0.07631404052424763
### iteration step :  110  rmse :  0.06676750299360984
### iteration step :  120  rmse :  0.05980463031599824
### iteration step :  130  rmse :  0.054654276584061495
### iteration step :  140  rmse :  0.050794278910791295
### iteration step :  150  rmse :  0.047863899317952915
### iteration step :  160  rmse :  0.04561028209978325
### iteration step :  170  rmse :  0.04385422600656379
### iteration step :  1

In [ ]:
ratings_pred_matrix_p = pd.DataFrame(data=pred_matrix, index= data_pivot.index,
                                   columns = data_pivot.columns)

In [ ]:
ratings_pred_matrix_p.to_csv(path + 'ratings_pred_matrix_p.csv' )

In [ ]:
all_predictions = pd.DataFrame(ratings_pred_matrix_p.unstack()).reset_index()
all_predictions = all_predictions[['userID', 'itemID', 0]]
all_predictions.rename({0:'prediction'}, axis=1, inplace=True)

In [ ]:
all_predictions['timestamp'] = 1475971200

In [ ]:
test.to_csv(path + 'mf_test.csv')
all_predictions.to_csv(path + 'mf_all_predictions.csv')

# mf default

In [ ]:
TOP_K = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

In [ ]:
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.016408
NDCG:	0.031498
Precision@K:	0.021136
Recall@K:	0.043550


# category

In [ ]:
def weighted_ensemble(a,b):
    final_score=a*(a/(a+b))+b*(b/(a+b))
    return final_score

In [ ]:
test = pd.read_csv(path + 'mf_test.csv', index_col=0)

In [ ]:
category_sim_mean = pd.DataFrame(np.mean(category_sim))

In [ ]:
ratings_pred_matrix_p = pd.read_csv(path + 'ratings_pred_matrix_p.csv', index_col = 0)


In [ ]:
ratings_pred_matrix_p_we = weighted_ensemble(ratings_pred_matrix_p.T,category_sim_mean.values)
ratings_pred_matrix_p_we = ratings_pred_matrix_p_we.T

In [ ]:
all_predictions = pd.DataFrame(ratings_pred_matrix_p_we.unstack()).reset_index()
all_predictions = all_predictions[['userID', 'level_0', 0]]
all_predictions.rename({0:'prediction', 'level_0':'itemID'}, axis=1, inplace=True)

In [ ]:
all_predictions['timestamp'] = 1475971200

In [ ]:
TOP_K = 10

In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.015506
NDCG:	0.031121
Precision@K:	0.021282
Recall@K:	0.043070


# description

In [ ]:
def weighted_ensemble(a,b):
    final_score=a*(a/(a+b))+b*(b/(a+b))
    return final_score

In [ ]:
test = pd.read_csv(path + 'mf_test.csv', index_col=0)

In [ ]:
description_sim_mean = pd.DataFrame(np.mean(description_sim))

In [ ]:
ratings_pred_matrix_p = pd.read_csv(path + 'ratings_pred_matrix_p.csv', index_col = 0)


In [ ]:
ratings_pred_matrix_p_we = weighted_ensemble(ratings_pred_matrix_p.T,description_sim_mean.values)
ratings_pred_matrix_p_we = ratings_pred_matrix_p_we.T

In [ ]:
all_predictions = pd.DataFrame(ratings_pred_matrix_p_we.unstack()).reset_index()
all_predictions = all_predictions[['userID', 'level_0', 0]]
all_predictions.rename({0:'prediction', 'level_0':'itemID'}, axis=1, inplace=True)

In [ ]:
all_predictions['timestamp'] = 1475971200

In [ ]:
TOP_K = 10

In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.016474
NDCG:	0.032077
Precision@K:	0.020952
Recall@K:	0.042821
